<center><h1>Shah_Shrey_HW3</h1></center>

Name: Shrey Shah<br>Github Username: shreyshah97<br>USC ID: 5061063668

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [1]:
import numpy as np
import os
import pandas as pd
from scipy.stats import bootstrap

import warnings
warnings.filterwarnings('ignore')

### (a) Get the AReM Data Set

Data is present in the ../data folder
<br>dataset4.csv in bending2 and dataset9.csv in cycling have been manually modified

### (b) Test and Train Data

In [2]:
def train_test_split(file_path):
    
    df_train = pd.DataFrame()
    df_test = pd.DataFrame()
    
    folders = os.listdir(file_path)

    for folder in folders:
        if 'pdf' in folder:
            continue
        path = os.path.join(file_path, folder)
        files = os.listdir(path)

        if folder == 'bending1' or folder == 'bending2':
            for file in files:
                data = pd.read_csv(os.path.join(file_path, folder, file), skiprows=4, usecols=range(7))
                if file == 'dataset1.csv' or file == 'dataset2.csv':
                    df_test = pd.concat([df_test, data], ignore_index = True)
                else:
                    df_train = pd.concat([df_train, data], ignore_index = True)
                
        else:
            for file in files:
                data = pd.read_csv(os.path.join(file_path, folder, file), skiprows=4 ,usecols=range(7))
                if file == 'dataset1.csv' or file == 'dataset2.csv' or file == 'dataset3.csv':
                    df_test = pd.concat([df_test, data], ignore_index = True)
                else:
                    df_train = pd.concat([df_train, data], ignore_index = True)
    
    return df_train, df_test
    

In [3]:
FILEPATH = "../data/AReM/"
df_train, df_test = train_test_split(FILEPATH)
df_train.head()

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,0,42.00,0.71,21.25,0.43,30.00,0.00
1,250,41.50,0.50,20.25,1.48,31.25,1.09
2,500,41.50,0.50,14.25,1.92,33.00,0.00
3,750,40.75,0.83,15.75,0.43,33.00,0.00
4,1000,40.00,0.71,20.00,2.74,32.75,0.43


### (c) Feature Extraction

#### i. Research

1. Maximum - Maximum of a time series data values.
2. Minimum - Minimum of a time series data values.
3. Mean - Arithmetic mean of the data.
4. Median - The value of 50th percentile of the time series data.
5. Standard deviation - Measure of variability of data values.
6. First quartile - The value of 25th percentile of the time series data.
7. Third quartile - The value of 75th percentile of the time series data.
8. Interquartile Range - The difference between 75th and 25th percentiles of the time serives values.
9. Cross-Correlations - Measures the similarity between two time series by computing the correlation between them at different lags.
10. Auto-correlations - Measure of a correlation between a time series and a lagged version of itself.
11. Skewness - Measure of asymmetry of the distribution of data.
12. Orders of the autoregressive (AR), integrated (I) and moving average (MA) part of an estimated ARIMA model

#### ii. Extraction

In [4]:
def get_data(file_path):
    
    folders = os.listdir(file_path)
    data = []
    
    for folder in folders:
        if 'pdf' in folder:
            continue
        path = os.path.join(file_path, folder)
        files = os.listdir(path)
        
        for file in files:
            data.append(pd.read_csv(os.path.join(path, file), skiprows=4 ,usecols=range(7)))
                
    return data

In [5]:
time_series_strings = ['min', 'max', 'mean', 'median', 'standard deviation', '1st quartile', '3rd quartile' ]
columns = []
for i in range (1,7):
    columns.append([a+str(i) for a in time_series_strings])
columns = [item for sublist in columns for item in sublist]
df = pd.DataFrame(columns = columns)

data = get_data(FILEPATH)
for d in data:
    summary = d.describe()
    row = []
    for i in range(1,7):
        row.append(summary.iloc[3,i])
        row.append(summary.iloc[7,i])
        row.append(summary.iloc[1,i])
        row.append(summary.iloc[5,i])
        row.append(summary.iloc[2,i])
        row.append(summary.iloc[4,i])
        row.append(summary.iloc[6,i])
    df = df.append(pd.DataFrame([row], columns = columns), ignore_index = True)
df.index + 1
df

,min1,max1,mean1,median1,standard deviation1,1st quartile1,3rd quartile1,min2,max2,mean2,...,standard deviation5,1st quartile5,3rd quartile5,min6,max6,mean6,median6,standard deviation6,1st quartile6,3rd quartile6
0,37.25,45.00,40.624792,40.500,1.476967,39.25,42.0000,0.0,1.30,0.358604,...,2.188449,33.0000,36.00,0.0,1.92,0.570583,0.430,0.582915,0.00,1.3000
1,38.00,45.67,42.812812,42.500,1.435550,42.00,43.6700,0.0,1.22,0.372437,...,1.995255,32.0000,34.50,0.0,3.11,0.571083,0.430,0.601010,0.00,1.3000
2,35.00,47.40,43.954500,44.330,1.558835,43.00,45.0000,0.0,1.70,0.426250,...,1.999604,35.3625,36.50,0.0,1.79,0.493292,0.430,0.513506,0.00,0.9400
3,33.00,47.75,42.179812,43.500,3.670666,39.15,45.0000,0.0,3.00,0.696042,...,3.849448,30.4575,36.33,0.0,2.18,0.613521,0.500,0.524317,0.00,1.0000
4,33.00,45.75,41.678063,41.750,2.243490,41.33,42.7500,0.0,2.83,0.535979,...,2.411026,28.4575,31.25,0.0,1.79,0.383292,0.430,0.389164,0.00,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,20.75,46.25,34.763333,35.290,4.742208,31.67,38.2500,0.0,12.68,4.223792,...,3.174681,14.2500,18.33,0.0,9.39,3.288271,3.270,1.647528,2.05,4.3050
84,21.50,51.00,34.935812,35.500,4.645944,32.00,38.0625,0.0,12.21,4.115750,...,3.192058,14.2375,18.25,0.0,10.21,3.280021,3.015,1.700918,2.12,4.5000
85,18.33,47.67,34.333042,34.750,4.948770,31.25,38.0000,0.0,12.48,4.396958,...,3.000493,13.7500,18.00,0.0,8.01,3.261583,2.980,1.617290,2.05,4.3200
86,18.33,45.75,34.599875,35.125,4.731790,31.50,38.0000,0.0,15.37,4.398833,...,2.905688,14.0000,18.25,0.0,8.86,3.289542,3.015,1.680170,2.12,4.2600


#### iii. Standard Deviation

In [6]:
df.describe().loc[['std']].T

,std
min1,9.569975
max1,4.394362
mean1,5.335718
median1,5.440054
standard deviation1,1.772153
1st quartile1,6.153590
3rd quartile1,5.138925
min2,0.000000
max2,5.062729
mean2,1.574164


In [7]:
confidence_intervals = pd.DataFrame()
for i in range(len(df.columns)):
    sample_data = (df.iloc[:, i],)
    res = bootstrap(sample_data, np.std, confidence_level = 0.9, random_state = 1, method = 'percentile')
    temp = pd.DataFrame({'Feature': df.columns[i], 'Lower Bound': res.confidence_interval.low, 'Upper Bound': res.confidence_interval.high}, index=[i+1])
    confidence_intervals = pd.concat([confidence_intervals, temp], ignore_index = True)
confidence_intervals

,Feature,Lower Bound,Upper Bound
0,min1,8.208212,10.718897
1,max1,3.302124,5.277910
2,mean1,4.683108,5.864954
3,median1,4.774987,5.983471
4,standard deviation1,1.561586,1.930514
5,1st quartile1,5.548767,6.619312
6,3rd quartile1,4.299096,5.819304
7,min2,0.000000,0.000000
8,max2,4.598099,5.372570
9,mean2,1.388105,1.698038


#### iv. Select Features

From the above summary data, according to me, the 3 most important time domain features:
1. Mean: Mean of the points can inform us about the central tendency of the data, taking various outliers into consideration.
2. Median: Since the data has too many outliers which can make the data skewed, median could be a good feature to have which is not affected much by outliers.
3. Standard deviation: Standard deviation tells us that roughly how many datapoints are away from the mean. It gives an insight into the underlying data distribution.


## 2. ISLR 3.7.4

### (a) Linear Train

<b>RSS for cubic regression would be smaller as compared to RSS for linear regression.</b><br>
The cubic regression model will have better flexibility and will better fit to the training data, hence lead to a lower RSS value as compared to the RSS value for linear regression.

### (b) Linear Test

<b>RSS for linear regression would be smaller as compared to RSS for cubic regression.</b><br>
The cubic regression model will overfit the training data and will result in poorer predictions for the test data as compared to the predictions of the linear regression model.

### (c) Not Linear Train

<b>RSS for cubic regression would be smaller as compared to RSS for linear regression.</b><br>
The principle remains the same, the cubic regression model has more flexibility and will be able to better fit the training data and hence will have a lower RSS value as compared to the linear regression model.

### (d) Not Linear Testing

<b>The information given here is insufficient to say anything</b><br>
For test data, the RSS would depend on the relationship of the data, if the data is closer to linear, than linear will give lower RSS than cubic regression.<br>
Otherwise, if the data is far from linear, cubic will give lower RSS than linear regression.

### References

https://stats.stackexchange.com/questions/50807/features-for-time-series-classification<br>